In [1]:
import tensorflow as tf
import os
import pandas as pd
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from cosme_model import COSMELayer
from global_funcs import load_model_config, load_program_config

In [2]:
model_config = load_model_config()
model_config

{'preblock_filters': 64,
 'preblock_kernel_sizes': [3, 5, 7, 9, 11, 13],
 'preblock_pool_size': 2,
 'idblock_kernel_sizes': [3, 5, 7, 9, 11, 13],
 'idblock_filters': [64, 128, 256, 512],
 'idblock_activation': 'gelu',
 'idblock_avg_pool_size': 2,
 'last_activation': 'softmax'}

In [3]:
preblock_filters = model_config['preblock_filters']
preblock_kernel_sizes = model_config['preblock_kernel_sizes']
preblock_pool_size = model_config['preblock_pool_size']
idblock_kernel_sizes = model_config['idblock_kernel_sizes']
idblock_filters = model_config['idblock_filters']
idblock_activation = model_config['idblock_activation']
idblock_avg_pool_size = model_config['idblock_avg_pool_size']
last_activation = model_config['last_activation']

In [4]:
type(idblock_avg_pool_size)

int

In [5]:
configs = load_program_config()

In [6]:
output_dir = configs['output_dir']
project_name = configs['project_name']

In [7]:
n_classes = pd.read_csv(f"{output_dir}/{project_name}/data/unq_labels.csv" ).shape[0]
n_classes

855

In [8]:
input_layer = tf.keras.layers.Input(shape=(1,150,), dtype=tf.float32)
cosme_layer = COSMELayer(
                preblock_filters=preblock_filters,
                preblock_kernel_sizes=preblock_kernel_sizes,
                preblock_pool_size=preblock_pool_size,
                idblock_kernel_sizes=idblock_kernel_sizes,
                idblock_filters = idblock_filters,
                idblock_activation=idblock_activation,
                idblock_avg_pool_size=idblock_avg_pool_size,
                last_activation=last_activation,
                n_classes=n_classes,
                )
output_layer = cosme_layer(input_layer)

2022-05-23 05:05:36.433520: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-23 05:05:36.454550: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-05-23 05:05:36.454566: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-05-23 05:05:36.454921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [9]:
cosme_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
cosme_model.compile(optimizer='adam' , loss = tf.keras.losses.sparse_categorical_crossentropy , metrics=['sparse_categorical_accuracy'])

In [10]:
tf.keras.utils.plot_model(cosme_model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [14]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [16]:
os.environ['TENSORBOARD_BINARY'] = '/home/jcosme/miniconda3/envs/tf/bin/tensorboard'

In [17]:
%tensorboard --logdir tf_logs